This file useses the purchases of the four FAs that we could find (2239-5-LR21, 2239-8-LR23, 2239-4-LR17 and ID 2239-20-LP13 ) and estimates a demand model. 

    'IDProductoCM' -> j 
    To create product characteristics: 'Tipo de Producto', 'Marca', 'Nombre Producto ONU'
    'Precio Unitario'-> net not including taxes, just changes the scale of the price parameter 
    'Rut Unidad de Compra' -> i 
    To create the groups (k):  'Región Unidad de Compra', 'Sector'
     'year' -> t 

     'Modelo'-> to do the match with the product characteristics. 
        
Variables not directly paired but which could be useful: 
    'Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco', 'CodigoOC', 'Fecha Envío OC', 'Cantidad', 'Rut Proveedor', 
    'Nombre Proveedor Sucursal', 'Orgcode_Comprador', 'Entcode_Comprador', 


In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pandasgui import show
import socket
import numpy as np
from unidecode import unidecode
import unicodedata
import re
from difflib import SequenceMatcher
import pyblp


In [35]:
dest_path = os.path.abspath(os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'joined_cleaned_transac.csv'))
print(f"Absolute path of dest_path: {dest_path}")
transac_df = pd.read_csv(dest_path) 



rename_dict = {
    'Rut Unidad de Compra': 'i',
    'IDProductoCM': 'product_ids',
    'Tipo de Producto': 'x1',
    'Marca' : 'x2',
    'Nombre Proveedor Sucursal' : 'x3',
    'Precio Unitario': 'prices',
}

transac_df.rename(columns=rename_dict, inplace=True)

Absolute path of dest_path: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\interm_data\yearly_data\Transacciones\joined_cleaned_transac.csv


In [36]:
# create the k groups based on 'Region Unidad de Compra' and 'Sector' 
transac_df['k'] = transac_df.groupby(['Sector', 'Región Unidad de Compra']).ngroup()

#define the market 
transac_df['month'] = pd.to_datetime(transac_df['Fecha Envío OC']).dt.month
transac_df['trimester'] = ((transac_df['month'] - 1) // 3) + 1
transac_df['semester'] = ((transac_df['month'] - 1) // 6) + 1


transac_df['market_ids3'] = transac_df.groupby(['year', 'month', 'Región Unidad de Compra']).ngroup()
transac_df['market_ids2'] = transac_df.groupby(['year', 'trimester', 'Región Unidad de Compra']).ngroup()
transac_df['market_ids'] = transac_df.groupby(['year', 'semester', 'Región Unidad de Compra']).ngroup()
#transac_df = transac_df.drop('Fecha Envío OC', axis=1)


#create a number of consumers at the market level 
region_counts = transac_df.groupby('Región Unidad de Compra')['i'].nunique()
transac_df['N_i'] = transac_df['Región Unidad de Compra'].map(region_counts)

#drop obs with missing values 
transac_df = transac_df.dropna(subset=['x1', 'x2', 'x3'])


transac_df
#

,product_ids,Modelo,x1,x2,Nombre Producto ONU,prices,i,Región Unidad de Compra,Sector,year,...,Puntaje Precio_y,exact_merge_var,k,month,trimester,semester,market_ids3,market_ids2,market_ids,N_i
0,1536899,H-1 MB 2.5 CRDI 6M/T GLS 10S AC 2AB ABS,MINIBUS,HYUNDAI,Minibuses,17098600.0,69.071.700-K,Metropolitana,Municipalidades,2018,...,NaN,NaN,48,2,1,1,6,11,12,216
1,1536914,SANTA FE DM WGN 2.4 6A/T 4WD GLS FULL PE,SUV,HYUNDAI,Automóviles,19740000.0,69.073.500-8,Valparaíso,Municipalidades,2018,...,NaN,NaN,50,2,1,1,8,13,14,99
2,1536916,SANTA FE DM WGN 2.4 6A/T GLS PE,SUV,HYUNDAI,Automóviles,14921513.0,65.154.016-k,Metropolitana,"Gob. Central, Universidades",2018,...,NaN,NaN,24,2,1,1,6,11,12,216
3,1536916,SANTA FE DM WGN 2.4 6A/T GLS PE,SUV,HYUNDAI,Automóviles,14921513.0,65.154.021-6,Metropolitana,"Gob. Central, Universidades",2018,...,NaN,NaN,24,2,1,1,6,11,12,216
4,1536979,NEW MAHINDRA SCORPIO SUV 4X4 ABS 2AB,SUV,MAHINDRA,Automóviles,10497983.0,61.955.100-1,Araucanía,Salud,2018,...,NaN,NaN,86,2,1,1,0,1,1,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4277,2049176,V700 RAPID 1.4L 2024,FURGÓN,RAM,Mini-furgonetas o furgonetas,12410800.0,65.154.398-3,Tarapacá,"Gob. Central, Universidades",2024,...,NaN,NaN,25,8,3,2,900,388,215,26
4278,2054456,D60 ELITE 7DCT 4X2 AT 2024,SUV,MAXUS,Automóviles,16403950.0,69.130.600-3,Maule,Municipalidades,2024,...,NaN,NaN,47,8,3,2,898,386,213,56
4279,2048705,T8 CE COMFORT 2.0T 4X2 MT 2024,CAMIONETA,JAC,Automóviles,15541506.0,69.071.800-6,Metropolitana,Municipalidades,2024,...,NaN,NaN,48,8,3,2,899,387,214,216
4280,2048790,T8 PRO LUXURY 4X4 MT 2024,CAMIONETA,JAC,Automóviles,18481640.0,69.071.800-6,Metropolitana,Municipalidades,2024,...,NaN,NaN,48,8,3,2,899,387,214,216


In [37]:
# Count unique combinations of x1, x2, x3
unique_combinations = transac_df.groupby(['x1', 'x2', 'x3']).size().reset_index().shape[0]

# Count individual unique values
x1_unique = transac_df['x1'].nunique()
x2_unique = transac_df['x2'].nunique()
x3_unique = transac_df['x3'].nunique()

print(f"Number of unique combinations (x1, x2, x3): {unique_combinations}")
print(f"Number of unique x1 values: {x1_unique}")
print(f"Number of unique x2 values: {x2_unique}")
print(f"Number of unique x3 values: {x3_unique}")

Number of unique combinations (x1, x2, x3): 381
Number of unique x1 values: 10
Number of unique x2 values: 41
Number of unique x3 values: 65


## Demand with micro-data

In [494]:
#calculate the market shares. 
observed_shares = product_data[['product_ids', 'market_ids', 'shares_real']].copy()
show(observed_shares)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [495]:
rename_dict = {
    'product_ids': 'j',
    'market_ids': 't',
}

transac_df.rename(columns=rename_dict, inplace=True)


In [496]:
vars_list = ['i', 'j', 'k', 't', 'x1', 'x2', 'x3', 'prices']
show(transac_df[vars_list])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Reduce the dimensionality of x1,x2 and x3. 

In [497]:
for col in ['x1', 'x2', 'x3']:
    # Calculate the value counts for the current column
    col_counts = transac_df[col].value_counts()
    print( col_counts)
    # Identify values that occur in less than 10% of the observations
    values_to_replace = col_counts[col_counts < len(transac_df) * 0.1].index
    
    # Replace these values with 'Other'
    transac_df[col] = transac_df[col].replace(values_to_replace, 'Other')


    # Count the number of unique values for 'x1', 'x2', and 'x3'
    x1_unique_values = transac_df['x1'].nunique()
    x2_unique_values = transac_df['x2'].nunique()
    x3_unique_values = transac_df['x3'].nunique()

    print(f"Number of unique values in 'x1,x2,x3': {x1_unique_values, x2_unique_values, x3_unique_values}")

x1
CAMIONETA               2189
SUV                      908
MINIBUS                  380
SEDÁN                    254
AMBULANCIAS              184
VEHÍCULOS POLICIALES     119
CAMIÓN LIVIANO           103
FURGÓN                    95
HATCHBACK                 41
CARGO                      8
Name: count, dtype: int64
Number of unique values in 'x1,x2,x3': (3, 41, 65)
x2
HYUNDAI          676
TOYOTA           651
NISSAN           648
MAXUS            339
-                303
CHEVROLET        261
MITSUBISHI       208
GREAT WALL       141
JMC              139
JAC               91
MAZDA             83
KIA               73
FORD              71
VOLKSWAGEN        67
SUZUKI            66
PEUGEOT           49
SSANGYONG         45
RAM               44
RENAULT           40
KIA MOTORS        39
CITROEN           37
CHANGAN           37
MAHINDRA          27
HONDA             26
GREATWALL         25
CHERY             22
FIAT              18
FOTON             13
SUBARU             7
MG                

create vector of params. 

In [498]:
import numpy as np
import pandas as pd

# --------------------------------------------------------
# 1) Identify the unique groups and category levels
# --------------------------------------------------------
k_groups = transac_df["k"].unique()
x1_cats  = transac_df["x1"].unique()
x2_cats  = transac_df["x2"].unique()
x3_cats  = transac_df["x3"].unique()

Nk = len(k_groups)
N1 = len(x1_cats)
N2 = len(x2_cats)
N3 = len(x3_cats)

# --------------------------------------------------------
# 2) Define product = (x1, x2, x3), then find unique (product, t) combos
#    because xi_jt is one parameter per product‐market pair
# --------------------------------------------------------
# Make an integer product_id from the combination (x1,x2,x3).
#   .ngroup() assigns a 0‐based ID to each distinct triple
transac_df = transac_df.copy()
transac_df["product_id"] = (
    transac_df.groupby(["x1","x2","x3"]).ngroup()
)

# Get all unique (product_id, t) pairs
unique_jt = (
    transac_df[["product_id","t"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
n_jt = len(unique_jt)   # number of distinct product‐market combos

# --------------------------------------------------------
# 3) Work out total parameter dimensions
# --------------------------------------------------------
# eta_k covers all categories of x1,x2,x3 for each group
dim_eta   = Nk * (N1 + N2 + N3)
dim_alpha = Nk
dim_xi    = n_jt
total_dim = dim_eta + dim_alpha + dim_xi

print(f"Nk={Nk}, N1={N1}, N2={N2}, N3={N3}, # of (j,t)={n_jt}")
print(f"Total number of parameters = {total_dim}")

# --------------------------------------------------------
# 4) Create a single (initial) parameter vector of zeros
#    plus a dictionary that maps each logical parameter
#    to an index in that vector.
# --------------------------------------------------------
theta_init = np.zeros(total_dim)

# A dictionary to keep track of where each parameter lives in theta_init
param_index = {}
idx = 0

# --- (A) The eta_k parameters for x1, x2, x3
# We’ll store them grouped by (feature_name, category_value, group_k)
for col_name, cat_list in zip(["x1","x2","x3"], [x1_cats, x2_cats, x3_cats]):
    for k_val in k_groups:
        for cat_val in cat_list:
            # param_index key describing which param this is
            param_index[("eta", k_val, col_name, cat_val)] = idx
            idx += 1

# --- (B) The alpha_k parameters
for k_val in k_groups:
    param_index[("alpha", k_val)] = idx
    idx += 1

# --- (C) The xi_{j,t} parameters
# For each distinct product_id, market t pair
for row in unique_jt.itertuples():
    # row has row.Index, row.product_id, row.t
    param_index[("xi", row.product_id, row.t)] = idx
    idx += 1

assert idx == total_dim, "Indexing mismatch—somehow dimension doesn't match"

# --------------------------------------------------------
# Done!  Now param_index tells you where in theta_init each
# logical parameter sits.  For example:
#   param_index[("alpha", 2)]        -> returns the position
#   param_index[("eta", 1, "x1", "SUV")] -> returns the position
#
# You can fill in theta_init[...] with starting guesses if you like.
# --------------------------------------------------------

print("Created param_index dictionary with size =", len(param_index))
print("theta_init shape =", theta_init.shape)


Nk=102, N1=3, N2=4, N3=4, # of (j,t)=1935
Total number of parameters = 3159
Created param_index dictionary with size = 3159
theta_init shape = (3159,)


: 

### GMM with microdata

In [ ]:
def compute_predicted_shares(df, params, Pi, group_col="k"):
    """
    df: one row per (market t, product j, buyer i) or a merged df that
        allows you to sum over the set of potential choices in market t.
        Must contain columns: 't', 'j', 'p', 'x1', 'x2', 'x3', group_col, etc.
    
    params: dictionary or array with your parameters, e.g.:
        {
          'eta': {...},      # mapping from x's to utility
          'alpha': {...},    # price coefficient by group
          'beta_u': {...},   # buyer-level random effect
          ...
        }
    Pi: dict or array of population weights by group k, e.g. Pi[k] = share of group k

    group_col: name of the column that indicates buyer's group (k)
    
    Returns: DataFrame with columns [t, j, predicted_share], i.e. s_jt(\theta).
    """
    
    # ---- STEP A: Compute the utility "inside" exponent for each row in df.
    # For example:
    #    utility = eta_k * x_{jt} - alpha_k * p_{jt} + beta^u * (v_i x_{tj}) + xi_{jt}
    # You will need to figure out how to map each group k to the right coefficients.
    # For now, let's do a toy version that lumps x1,x2,x3 into one 'quality' measure.
    
    # We'll pretend 'eta_k' depends only on the group.  Something like:
    #   utility = params["eta"][df[group_col]] * df["quality"] - params["alpha"][df[group_col]]*df["p"]
    
    # This is obviously very simplified. Real BLP code has more structure.
    
    df = df.copy()
    
    # Example: define a numeric "quality" measure from x1,x2,x3
    # purely for demonstration. In reality you would encode them properly:
    df["quality"] = (
        df["x1"].astype("category").cat.codes 
        + df["x2"].astype("category").cat.codes 
        + df["x3"].astype("category").cat.codes
    )

    # A simple group-based alpha, eta:
    # If you store them in a dict keyed by group, e.g. alpha[1], alpha[2], ...
    alpha_k = df[group_col].map(params["alpha"])  # maps each row's group to the alpha param
    eta_k   = df[group_col].map(params["eta"])
    # Then define inside exponent:
    df["util"] = eta_k*df["quality"] - alpha_k*df["p"]  
    # we’re ignoring random buyer effect or xi_{jt} for now, just to illustrate.
    
    # ---- STEP B: For each market t, we want to sum over j in the denominator.
    # We can do a "log-sum-exp" approach or we can do the naive approach:
    
    # 1) exponentiate
    df["exp_util"] = np.exp(df["util"])
    
    # 2) sum across *all products j in that market t*, for each buyer i:
    #    we do groupby(["t","i"]) then sum the exponentiated utilities
    df["denominator"] = df.groupby(["t","i"])["exp_util"].transform("sum")
    
    # 3) The conditional choice probability for each row:
    df["choice_prob_i"] = df["exp_util"] / df["denominator"]
    
    # ---- STEP C: Now average across buyers in the same group k within (j,t),
    # then weight by Pi_k to get the final predicted share s_jt.
    # So for each (t, j, k), we do mean(choice_prob_i), then multiply by Pi_k,
    # then sum over k.
    
    # first do unweighted average by group
    grouped = df.groupby(["t","j", group_col], as_index=False)["choice_prob_i"].mean()
    grouped.rename(columns={"choice_prob_i": "mean_choice_prob_k"}, inplace=True)
    
    # now multiply that mean by Pi_k
    grouped["group_weight"] = grouped[group_col].map(Pi)
    grouped["pred_part"] = grouped["mean_choice_prob_k"] * grouped["group_weight"]
    
    # sum across groups k to get the final predicted share for each (j,t)
    predicted_shares = (
        grouped
        .groupby(["t","j"], as_index=False)["pred_part"]
        .sum()
        .rename(columns={"pred_part": "s_jt_pred"})
    )
    
    return predicted_shares

# EXAMPLE USAGE:
# Suppose you have K=3 groups (k=1,2,3),
# Pi = {1: 0.2, 2: 0.3, 3: 0.5}
# params = {
#    "alpha": {1: 0.5, 2: 0.8, 3: 1.1},
#    "eta":   {1: 2.0, 2: 2.2, 3: 2.5}
# }
#
# Then do:
#
# pred_df = compute_predicted_shares(transac_df, params, Pi, group_col="k")
# pred_df.head()



1. compute the likelihood given a parameter vector 
I only sum over the observations we see 

In [455]:
unique_k = transac_df['k'].unique()

# Here we initialize ηₖ and αₖ as 1.0 for each unique group k:
eta_k_dict   = {k_val: 1.0 for k_val in unique_k}
alpha_k_dict = {k_val: 1.0 for k_val in unique_k}

# 2. Create param_guess for the product-time fixed effects ξⱼₜ
unique_jt = transac_df[['j', 't']].drop_duplicates()
xi_dict = {}
for _, row in unique_jt.iterrows():
    j_val = row['j']
    t_val = row['t']
    xi_dict[(j_val, t_val)] = 0.0

# Finally, assemble everything into a single dictionary param_guess
param_guess = {
    'eta_k':   eta_k_dict,       # dict: k -> 1.0
    'alpha_k': alpha_k_dict,     # dict: k -> 1.0
    'beta_u':  1.0,              # scalar
    'sigma':   1.0,              # scalar
    'xi':      xi_dict           # dict: (j, t) -> 0.0
}

# Now param_guess is ready to be used, for instance, 
# in your log-likelihood evaluation or optimization routine.
print(param_guess)


{'eta_k': {48: 1.0, 50: 1.0, 24: 1.0, 86: 1.0, 18: 1.0, 51: 1.0, 25: 1.0, 41: 1.0, 37: 1.0, 9: 1.0, 45: 1.0, 44: 1.0, 47: 1.0, 14: 1.0, 26: 1.0, 43: 1.0, 31: 1.0, 34: 1.0, 39: 1.0, 19: 1.0, 11: 1.0, 68: 1.0, 99: 1.0, 65: 1.0, 42: 1.0, 16: 1.0, 12: 1.0, 20: 1.0, 22: 1.0, 23: 1.0, 15: 1.0, 13: 1.0, 33: 1.0, 29: 1.0, 49: 1.0, 32: 1.0, 36: 1.0, 38: 1.0, 81: 1.0, 93: 1.0, 17: 1.0, 71: 1.0, 6: 1.0, 80: 1.0, 46: 1.0, 97: 1.0, 91: 1.0, 5: 1.0, 1: 1.0, 90: 1.0, 92: 1.0, 76: 1.0, 85: 1.0, 87: 1.0, 100: 1.0, 95: 1.0, 89: 1.0, 96: 1.0, 2: 1.0, 40: 1.0, 21: 1.0, 27: 1.0, 57: 1.0, 61: 1.0, 35: 1.0, 4: 1.0, 78: 1.0, 94: 1.0, 8: 1.0, 59: 1.0, 62: 1.0, 63: 1.0, 88: 1.0, 56: 1.0, 73: 1.0, 83: 1.0, 77: 1.0, 30: 1.0, 3: 1.0, 98: 1.0, 10: 1.0, 52: 1.0, 101: 1.0, 55: 1.0, 28: 1.0, 82: 1.0, 84: 1.0, 75: 1.0, 58: 1.0, 69: 1.0, 60: 1.0, 0: 1.0, 74: 1.0, 54: 1.0, 64: 1.0, 53: 1.0, 66: 1.0, 70: 1.0, 67: 1.0, 79: 1.0, 7: 1.0, 72: 1.0}, 'alpha_k': {48: 1.0, 50: 1.0, 24: 1.0, 86: 1.0, 18: 1.0, 51: 1.0, 25: 1.0, 41:

In [458]:
# Example: convert x1, x2, x3 to dummy variables
cat_dummies = pd.get_dummies(
    transac_df[['x1','x2','x3']], 
    prefix=['x1','x2','x3']
)
# Then concatenate these new columns onto transac_df
transac_df = pd.concat([transac_df, cat_dummies], axis=1)


In [461]:
show(transac_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [459]:
from collections import defaultdict

def log_likelihood(
    params,
    transac_df,
    product_chars=['x1','x2','x3'],
    n_draws=50,
    seed=1234
):
    """
    Compute the *simulated* log-likelihood of the observed purchase decisions
    in `transac_df`, under the random coefficients logit model:

        U_{ijt} = eta_k x_{jt} - alpha_k p_{jt} + beta^u * v_i * x_{jt} + xi_{jt} + epsilon_{ijt},

    where v_i ~ Normal(0, sigma^2) and epsilon_{ijt} is Type-I Extreme Value.

    Parameters
    ----------
    params : dict
        Dictionary of parameters with keys:
          - 'eta_k': dict or array with group k -> eta_k
          - 'alpha_k': dict or array with group k -> alpha_k
          - 'beta_u': float (the coefficient on v_i * x_{jt})
          - 'sigma': float (std dev of v_i)
          - 'xi': dict keyed by (j, t) -> xi_{jt}

    transac_df : pd.DataFrame
        Must contain columns: ['i','k','t','j','prices'] + product_chars

    product_chars : list of str
        Names of the product characteristic columns used in x_{jt}.

    n_draws : int
        Number of simulation draws for each individual from Normal(0, sigma^2).

    seed : int
        Random seed for reproducibility.

    Returns
    -------
    ll : float
        The simulated log-likelihood.
    """
    rng = np.random.default_rng(seed)

    # --- Parse the parameters from the dictionary ---
    eta_k   = params['eta_k']      # dictionary or array: group k -> eta_k
    alpha_k = params['alpha_k']    # dictionary or array: group k -> alpha_k
    beta_u  = params['beta_u']     # float
    sigma   = params['sigma']      # float
    xi_dict = params['xi']         # dict keyed by (j,t) -> xi_{jt}

    # --- For efficiency, group data by (t) so we can construct choice sets ---
    # We'll create a dictionary: t -> subset of transac_df
    # also we might store each product's data in that group for denominator.
    df_by_t = dict(tuple(transac_df.groupby('t')))

    # We also need to know the *distinct products* available for each t.
    # For simplicity, we assume all j that appear at time t are in the choice set:
    choice_sets = {
        t: df_by_t[t]['j'].unique()  
        for t in df_by_t
    }

    # Optional: Pre-generate v_i draws for each i (to keep them consistent across repeated observations of the same i).
    # We'll find all distinct i's:
    unique_is = transac_df['i'].unique()
    # For each i, draw n_draws from Normal(0, sigma^2)
    # Store in a dict: i -> array of shape (n_draws,)
    v_draws_dict = {
        i_val: rng.normal(loc=0.0, scale=sigma, size=n_draws)
        for i_val in unique_is
    }

    # Now we accumulate the log-likelihood
    ll = 0.0

    # We iterate over each row (i,j,t) in the data. Each row is the chosen product for buyer i at time t.
    for idx, row in transac_df.iterrows():
        i_val = row['i']
        k_val = row['k']    # group
        t_val = row['t']
        j_chosen = row['j'] # the chosen product

        # Price & product characteristics of chosen product j
        p_chosen = row['prices']
        x_chosen = row[product_chars].values  # array of x_{jt} for chosen product

        # Retrieve the parameter values for group k
        eta_k_val   = eta_k[k_val]     # \eta_{k}
        alpha_k_val = alpha_k[k_val]   # \alpha_{k}

        # xi_{j,t} for the chosen product
        xi_chosen = xi_dict[(j_chosen, t_val)]

        # The set of products that are available for time t_val:
        choice_set_t = choice_sets[t_val]

        # --- We do simulation over v_i draws ---
        v_i_draws = v_draws_dict[i_val]  # shape (n_draws,)

        # We'll accumulate probabilities across draws, then average
        prob_sum = 0.0

        # For each draw from v_i, compute the exponentiated utility, do the denominator sum, etc.
        for v_draw in v_i_draws:
            # Utility for the chosen product j_chosen:
            U_chosen = (eta_k_val * x_chosen.sum()
                        - alpha_k_val * p_chosen
                        + beta_u * v_draw * x_chosen.sum()
                        + xi_chosen)
            exp_U_chosen = np.exp(U_chosen)

            # Denominator: sum over all j' in choice_set_t
            denom = 0.0
            # We must compute U_{i f t} for each f in choice_set_t
            # For that, we can do the same formula:
            for j_alt in choice_set_t:
                # Subset the row(s) that correspond to j_alt in time t_val
                # to get its price & product characteristics. 
                # A quick way is to pick any row that has j=j_alt, t=t_val. 
                # (If there's one row per j-alt per t, we can do it by merging or caching.)
                # For speed, you might want a precomputed dictionary of (j,t) -> (price, x-vector).
                # We'll do a quick approach here:
                # WARNING: This can be slow in real code. Better to pre-map (j,t) -> (price, x-vector).
                mask_alt = (df_by_t[t_val]['j'] == j_alt)
                # We pick the *first* row that matches j_alt in time t_val:
                # (assuming each j_alt, t_val is unique or consistent).
                alt_row = df_by_t[t_val].loc[mask_alt].iloc[0]
                p_alt = alt_row['price']
                x_alt = alt_row[product_chars].values
                xi_alt = xi_dict[(j_alt, t_val)]

                U_alt = (eta_k_val * x_alt.sum()
                         - alpha_k_val * p_alt
                         + beta_u * v_draw * x_alt.sum()
                         + xi_alt)
                denom += np.exp(U_alt)

            prob_sum += exp_U_chosen / denom

        # The probability for this observation is the average across draws
        prob_ijt = prob_sum / n_draws

        # Accumulate log(prob)
        ll += np.log(prob_ijt + 1e-12)  # add small epsilon to avoid log(0)

    return ll


# Then we compute the log-likelihood:
ll_value = log_likelihood(
    param_guess,
    transac_df,
    product_chars=['x1', 'x2', 'x3'],
    n_draws=50,
    seed=123
)

print("Simulated Log-Likelihood at param_guess = ", ll_value)


TypeError: can't multiply sequence by non-int of type 'float'

2. calculate the likelihood given the vector 

3. minimize likelihood given the vector. 